In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
import math


In [2]:
# Global Functions
def sqScore(hits, shots, target):
    score = round(hits * math.sqrt((hits/shots)*100))
    print(f'Score: {score}')
    misses = shots - hits
    i = 0

    # Hits +1
    while target > score: 
        i += 1
        score = (hits + i) * math.sqrt(((hits + i)/shots)*100)
    else:
        print(f'Converted Hits: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0
    
    # Hits, Shots +1
    while target > score: 
        i += 1
        score = (hits + i) * math.sqrt(((hits + i)/(shots + i))*100)
    else:
        print(f'New Hits: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0

    # Shots -1
    while target > score: 
        i += 1
        score = (hits) * math.sqrt((hits/(shots - i))*100)
        if (shots - i) < hits:
            score = hits * math.sqrt((hits/shots)*100)
            break
    else:
        print(f'Fewer Misses: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0

    tableShots = [x for x in range(int(target/10), (round(1.25 * (target/10))))]
    tableHits = [math.ceil((target/10)**(2/3) * x**(1./3)) for x in tableShots]
    
    targetTable = pd.DataFrame({"Hits": tableHits, "Shots": tableShots})
    targetTable = targetTable.assign(Accuracy=100 * round(targetTable["Hits"]/targetTable["Shots"], 3)).set_index("Shots")
    print(targetTable)

In [3]:
username = 'ashjubilee'

url = "https://api.aimlab.gg/graphql"


In [11]:
graphql_query = """query GetProfile($username: String) {
        aimlabProfile(username: $username) {
            username
            id
        }
    }"""

variables = {
  "username": username,
  "user": {
                "id": ""
            }
}

data = {'query': graphql_query,
        "variables": variables}

headers = {
  'content-type': 'application/json'
}

response = requests.post(url=url, headers=headers, data=json.dumps(data))

pprint(response.json()['data']['aimlabProfile']['id'])


{'id': 'A369E0F06E148024', 'username': 'ashjubilee'}


In [8]:
graphql_query = """query GetAimlabProfileAgg($where: AimlabPlayWhere!) {
    aimlab {
      plays_agg(where: $where) {
        group_by {
          task_id
          task_name
        }
        aggregate {
          count
          avg {
            score
            accuracy
          }
          max {
            score
            accuracy
            created_at
          }
        }
      }
    }
  }"""

variables = {

    "where": {

        "is_practice": {
            "_eq": False
            },

        "score": {
            "_gt": 0
        },

        "user_id": {
            "_eq": "A369E0F06E148024"   
        }
    }
}

data = {'query': graphql_query,
        "variables": variables}

headers = {
  'content-type': 'application/json'
}

response = requests.post(url=url, headers=headers, data=json.dumps(data))

pprint(response.json())

{'data': {'aimlab': {'plays_agg': [{'aggregate': {'avg': {'accuracy': 38.68967819213867,
                                                          'score': 2309},
                                                  'count': 1,
                                                  'max': {'accuracy': 38.68968,
                                                          'created_at': '2023-07-23T22:31:20Z',
                                                          'score': 2309}},
                                    'group_by': {'task_id': 'CsLevel.VT '
                                                            'Empyrean.VT '
                                                            'Stead.RB67MZ',
                                                 'task_name': 'VT Steadytrack '
                                                              'Novice'}},
                                   {'aggregate': {'avg': {'accuracy': 57.370556500493265,
                                                       